In [2]:
!wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh 
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
!rm Miniconda3-latest-Linux-x86_64.sh
!conda install -y -q -c conda-forge openmm=7.6 python=3.7 pdbfixer 2>&1 1>/dev/null
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [3]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

prmtop = AmberPrmtopFile('ind_dhp_wt.prmtop')
inpcrd = AmberInpcrdFile('ind_dhp_wt.inpcrd')

platform = Platform.getPlatformByName("OpenCL")

min_steps = int(1000) # maximum minimizatiion steps
max_tolerance = int(10) # maximum minization energy tolerance
repsteps = int(1000) # report every n steps
mdsteps = int(1000000) # amount of MD steps to simulate
temp = 300 # temperature of the system

system = prmtop.createSystem(nonbondedMethod=NoCutoff,
                             nonbondedCutoff=1 * nanometer, rigidWater=True, ewaldErrorTolerance=0.0005)
integrator = LangevinIntegrator(temp * kelvin, 1 / picosecond, 2 * femtoseconds)
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
            simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
print('Starting minimization...')
simulation.reporters.append(StateDataReporter(stdout,
                                              10, progress=True, time=True,
                                              potentialEnergy=True, kineticEnergy=True,
                                              totalEnergy=True,temperature=True,
                                              remainingTime=True, speed=True,
                                              totalSteps=int(mdsteps+min_steps), separator='\t'))
if min_steps == '-':
  simulation.minimizeEnergy(tolerance=int(max_tolerance) * kilojoule / mole)
elif min_steps != '-':
  simulation.minimizeEnergy(tolerance=10 * kilojoule / mole, maxIterations=int(min_steps))
elif min_steps == '-' and max_tolerance =='-':
  simulation.minimizeEnergy()
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('minimized.pdb', 'w'))
simulation.saveState('minimized.xml')
simulation.saveCheckpoint('minimized.chk')

FileNotFoundError: [Errno 2] No such file or directory: 'ind_dhp_w.prmtop'

In [4]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

prmtop = AmberPrmtopFile('ind_dhp_wt.prmtop')
inpcrd = AmberInpcrdFile('ind_dhp_wt.inpcrd')

platform = Platform.getPlatformByName("OpenCL")

min_steps = int(1000) # maximum minimizatiion steps
max_tolerance = int(10) # maximum minization energy tolerance
repsteps = int(2000) # report every n steps
mdsteps = int(1000000) # amount of MD steps to simulate
temp = 300 # temperature of the system

system = prmtop.createSystem(nonbondedMethod=NoCutoff,
                             nonbondedCutoff=1 * nanometer, rigidWater=True, ewaldErrorTolerance=0.0005)
integrator = LangevinIntegrator(temp * kelvin, 1 / picosecond, 2 * femtoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
            simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
simulation.loadState('minimized.xml')
print('Reloading data...')
simulation.reporters.append(StateDataReporter(stdout,
                                              10, progress=True, time=True,
                                              potentialEnergy=True, kineticEnergy=True,
                                              totalEnergy=True,temperature=True,
                                              remainingTime=True, speed=True,
                                              totalSteps=int(mdsteps+min_steps), separator='\t'))


Reloading data...


In [5]:
platform = Platform.getPlatformByName("OpenCL")

simulation.reporters.append(DCDReporter('output.dcd', int(repsteps)))
simulation.reporters.append(PDBReporter('output.pdb', int(repsteps)))
simulation.reporters.append(StateDataReporter('ouptut.log',
                                              repsteps, step=True, time=True,
                                              potentialEnergy=True, kineticEnergy=True,
                                              totalEnergy=True,temperature=True,
                                              progress=True, volume=True,density=True,
                                              remainingTime=True, speed=True,
                                              totalSteps=int(mdsteps), separator='\t'))
simulation.reporters.append(CheckpointReporter('checkpnt.chk', int(repsteps)))
simulation.step(int(mdsteps))
print('MD finished!')

#"Progress (%)"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Kinetic Energy (kJ/mole)"	"Total Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
0.0%	0.020000000000000004	-1663737.0	7844.035921933953	-1655892.964078066	7.241012624834404	0	--
0.0%	0.04000000000000002	-1657507.75	14387.818673058733	-1643119.9313269414	13.28173120218963	0.12	16:16:30:40
0.0%	0.06000000000000004	-1650983.875	20462.848451557376	-1630521.0265484427	18.889732970685866	0.12	16:16:25:21
0.0%	0.08000000000000006	-1644986.125	26755.569426389724	-1618230.5555736104	24.698690563027824	0.12	16:16:23:01
0.0%	0.10000000000000007	-1638542.25	32300.630590711953	-1606241.619409288	29.817465935290112	0.12	16:16:23:43


KeyboardInterrupt: 